In [1]:
import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Tue_May__3_18:49:52_PDT_2022
Cuda compilation tools, release 11.7, V11.7.64
Build cuda_11.7.r11.7/compiler.31294372_0
torch:  2.0 ; cuda:  cu117
detectron2: 0.6


In [2]:
import os
import sys
import logging
import argparse

os.environ["DATASET"] = "../datasets"

pth = '/'.join(sys.path[0].split('/')[:-1])
sys.path.insert(0, pth)

from pprint import pprint
from PIL import Image
from copy import deepcopy
import numpy as np
np.random.seed(1)

home_dir = os.path.abspath(os.getcwd()+"/../")
sys.path.append(home_dir)
print(home_dir)

import warnings
warnings.filterwarnings(action='ignore')
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

import torch
from torchvision import transforms
from torch.nn import functional as F

from detectron2.data import MetadataCatalog
from detectron2.utils.colormap import random_color
from detectron2.structures import Boxes, ImageList, Instances, BitMasks, BoxMode

from hdecoder.BaseModel import BaseModel
from hdecoder import build_model

from utils.arguments import load_opt_command
from utils.misc import hook_metadata, hook_switcher, hook_opt
from utils.distributed import init_distributed
from utils.arguments import load_opt_from_config_files, load_config_dict_to_opt

/home/djjin/Mygit/X-Decoder


Invalid MIT-MAGIC-COOKIE-1 key

In [3]:
parser = argparse.ArgumentParser(description='Pretrain or fine-tune models for NLP tasks.')
parser.add_argument('--command', default="evaluate", help='Command: train/evaluate/train-and-evaluate')
parser.add_argument('--conf_files', nargs='+', help='Path(s) to the config file(s).')
parser.add_argument('--user_dir', help='Path to the user defined module for tasks (models, criteria), optimizers, and lr schedulers.')
parser.add_argument('--config_overrides', nargs='*', help='Override parameters on config with a json style string, e.g. {"<PARAM_NAME_1>": <PARAM_VALUE_1>, "<PARAM_GROUP_2>.<PARAM_SUBGROUP_2>.<PARAM_2>": <PARAM_VALUE_2>}. A key with "." updates the object in the corresponding nested dict. Remember to escape " in command line.')
parser.add_argument('--overrides', help='arguments that used to override the config file in cmdline', nargs=argparse.REMAINDER)

cmdline_args = parser.parse_args('')
# cmdline_args.conf_files = [os.path.join(home_dir, "configs/xdecoder/svlp_focalt_lang.yaml")]
cmdline_args.conf_files = [os.path.join(home_dir, "configs/hdecoder/vcoco.yaml")]
cmdline_args.overrides = ['WEIGHT', 'true', 'RESUME_FROM', '../checkpoints/xdecoder_focalt_best_openseg.pt'] 

opt = load_opt_from_config_files(cmdline_args.conf_files)
opt["base_path"] = "../"

keys = [cmdline_args.overrides[idx*2] for idx in range(len(cmdline_args.overrides)//2)]
vals = [cmdline_args.overrides[idx*2+1] for idx in range(len(cmdline_args.overrides)//2)]
vals = [val.replace('false', '').replace('False','') if len(val.replace(' ', '')) == 5 else val for val in vals]
types = []
for key in keys:
    key = key.split('.')
    ele = opt.copy()
    while len(key) > 0:
        ele = ele[key.pop(0)]
    types.append(type(ele))

config_dict = {x:z(y) for x,y,z in zip(keys, vals, types)}

load_config_dict_to_opt(opt, config_dict)
for key, val in cmdline_args.__dict__.items():
    if val is not None:
        opt[key] = val
opt = init_distributed(opt)

In [4]:
opt["WANDB"] = False
fix_param = opt['SOLVER'].get('FIX_PARAM',{})
fix_param

{'backbone': True, 'encoder': True}

In [5]:
from trainer import HDecoder_Trainer as Trainer
trainer = Trainer(opt)
trainer.train()

INFO:trainer.distributed_trainer:Setting SAVE_DIR as data/output/test
INFO:trainer.distributed_trainer:Using CUDA
INFO:trainer.distributed_trainer:Save config file to data/output/test/conf_copy.yaml
INFO:trainer.distributed_trainer:Base learning rate: 0.0001
INFO:trainer.distributed_trainer:Number of GPUs: 1
INFO:trainer.distributed_trainer:Gradient accumulation steps: 1
INFO:trainer.default_trainer:Imported base_dir at base_path ../
INFO:trainer.default_trainer:Pipeline for training: HDecoderPipeline
INFO:trainer.default_trainer:-------------------------------------------------------
INFO:trainer.default_trainer:Training on rank: 0
INFO:base_dir.pipeline.HDecoderPipeline:CDNHOI(
  (backbone): D2FocalNet(
    (patch_embed): PatchEmbed(
      (proj): Conv2d(3, 96, kernel_size=(7, 7), stride=(4, 4), padding=(3, 3))
      (norm): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
    )
    (pos_drop): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0): BasicLayer(
       

Dataset length: 5400
num of train samples: 2700


INFO:utils.model:Loaded backbone.layers.0.blocks.0.dw1.bias, Model Shape: torch.Size([96]) <-> Ckpt Shape: torch.Size([96])
INFO:utils.model:Loaded backbone.layers.0.blocks.0.dw1.weight, Model Shape: torch.Size([96, 1, 3, 3]) <-> Ckpt Shape: torch.Size([96, 1, 3, 3])
INFO:utils.model:Loaded backbone.layers.0.blocks.0.dw2.bias, Model Shape: torch.Size([96]) <-> Ckpt Shape: torch.Size([96])
INFO:utils.model:Loaded backbone.layers.0.blocks.0.dw2.weight, Model Shape: torch.Size([96, 1, 3, 3]) <-> Ckpt Shape: torch.Size([96, 1, 3, 3])
INFO:utils.model:Loaded backbone.layers.0.blocks.0.gamma_1, Model Shape: torch.Size([96]) <-> Ckpt Shape: torch.Size([96])
INFO:utils.model:Loaded backbone.layers.0.blocks.0.gamma_2, Model Shape: torch.Size([96]) <-> Ckpt Shape: torch.Size([96])
INFO:utils.model:Loaded backbone.layers.0.blocks.0.mlp.fc1.bias, Model Shape: torch.Size([384]) <-> Ckpt Shape: torch.Size([384])
INFO:utils.model:Loaded backbone.layers.0.blocks.0.mlp.fc1.weight, Model Shape: torch.Si

Evaluation start ...


INFO:base_dir.pipeline.HDecoderPipeline:Inference done 11/2473. Dataloading: 0.0143 s/iter. Inference: 0.1257 s/iter. Eval: 0.0003 s/iter. Total: 0.1403 s/iter. ETA=0:05:45
INFO:base_dir.pipeline.HDecoderPipeline:Inference done 47/2473. Dataloading: 0.0171 s/iter. Inference: 0.1219 s/iter. Eval: 0.0003 s/iter. Total: 0.1393 s/iter. ETA=0:05:37
INFO:base_dir.pipeline.HDecoderPipeline:Inference done 84/2473. Dataloading: 0.0172 s/iter. Inference: 0.1201 s/iter. Eval: 0.0003 s/iter. Total: 0.1376 s/iter. ETA=0:05:28
INFO:base_dir.pipeline.HDecoderPipeline:Inference done 120/2473. Dataloading: 0.0173 s/iter. Inference: 0.1213 s/iter. Eval: 0.0003 s/iter. Total: 0.1390 s/iter. ETA=0:05:26
INFO:base_dir.pipeline.HDecoderPipeline:Inference done 158/2473. Dataloading: 0.0174 s/iter. Inference: 0.1206 s/iter. Eval: 0.0003 s/iter. Total: 0.1382 s/iter. ETA=0:05:20
INFO:base_dir.pipeline.HDecoderPipeline:Inference done 196/2473. Dataloading: 0.0173 s/iter. Inference: 0.1197 s/iter. Eval: 0.0003 s

------------------------------------------------------------
               hold_obj: #GTs = 3608, AP = 0.0107
                  stand: #GTs = 4118, AP = 0.0222
              sit_instr: #GTs = 1916, AP = 0.0059
             ride_instr: #GTs = 0556, AP = 0.0001
                   walk: #GTs = 0597, AP = 0.0009
               look_obj: #GTs = 3347, AP = 0.0004
              hit_instr: #GTs = 0349, AP = 0.0006
                hit_obj: #GTs = 0349, AP = 0.0009
                eat_obj: #GTs = 0521, AP = 0.0010
              eat_instr: #GTs = 0521, AP = 0.0909
             jump_instr: #GTs = 0635, AP = 0.0004
              lay_instr: #GTs = 0387, AP = 0.0001
    talk_on_phone_instr: #GTs = 0285, AP = 0.0000
              carry_obj: #GTs = 0472, AP = 0.0001
              throw_obj: #GTs = 0244, AP = 0.0002
              catch_obj: #GTs = 0246, AP = 0.0000
              cut_instr: #GTs = 0269, AP = 0.0000
                cut_obj: #GTs = 0269, AP = 0.0026
                    run: #GTs = 0687, A

INFO:trainer.default_trainer:{'vcoco_val/vcoco': {'AP_hold_obj': 0.0106951871657754, 'AP_stand': 0.022228189910446797, 'AP_sit_instr': 0.005865102639296188, 'AP_ride_instr': 0.00011420740063956144, 'AP_walk': 0.0009072763563781528, 'AP_look_obj': 0.00042087542087542086, 'AP_hit_instr': 0.0005681818181818183, 'AP_hit_obj': 0.0009182736455463729, 'AP_eat_obj': 0.0010330578512396695, 'AP_eat_instr': 0.09090909090909091, 'AP_jump_instr': 0.0003852080123266564, 'AP_lay_instr': 0.00010215779974801077, 'AP_talk_on_phone_instr': 3.258390355164549e-05, 'AP_carry_obj': 6.343970056461333e-05, 'AP_throw_obj': 0.0001967729240456513, 'AP_catch_obj': 0.0, 'AP_cut_instr': 0.0, 'AP_cut_obj': 0.0025974025974025974, 'AP_run': 0.006493506493506493, 'AP_work_on_computer_instr': 0.018181818181818184, 'AP_ski_instr': 0.0006142506142506142, 'AP_surf_instr': 0.0, 'AP_skateboard_instr': 4.274052228918237e-05, 'AP_smile': 0.09184724491067855, 'AP_drink_instr': 0.0, 'AP_kick_obj': 0.0, 'AP_point_instr': 0.0, 'AP_

KeyboardInterrupt: 

: 